# Combining all the variables and their values for the patients into a single CSV

In [6]:
# Import libraries
import pandas as pd
import os
import psycopg2
import getpass
import pandas as pd

In [2]:
# Path to save the extracted values
export_dir = '../extracted_data'
if not os.path.isdir(export_dir):
    os.mkdir(export_dir)

In [3]:
# Create a database connection
user = 'dhruv.sharma'
host = 'localhost'
dbname = 'mimic'
schema = 'mimiciii'

## Loading the data

In [4]:
# Connect to the database
con = psycopg2.connect(dbname=dbname, user=user, host=host, 
                       password=getpass.getpass(prompt='Password:'.format(user)))
cur = con.cursor()
cur.execute('SET search_path to {}'.format(schema))

Password:········


## Combining the static variables

<b>Demographics:</b> Subject_ID, age, gender, weight

<b>Comorbidities:</b> Congestive Heart Failure, Cardiac arrhythmias, valvular disease, pulmonary circulation, peripheral vascular, hypertension, paralysis, neurological disorder, chronic pulmonary disease, diabetes, hypothyroidism, renal failure, liver diseases, peptic ulcer, AIDS, lymphoma, metastatic cancer, rheumatoid arthritis, coagulopathy, obesity, fluid electrolyte, anemias, alcohol abuse, drug abuse, psychosis, depression 

In [7]:
demographics_path = os.path.join(export_dir, 'demographics.csv')
comorbidities_path = os.path.join(export_dir, 'comorbidities.csv')

demographics = pd.read_csv(demographics_path)
comorbidities = pd.read_csv(comorbidities_path)